Preperation before running the test
1. Register the model/model.joblib in your workspace under the name sklearn-iris
2. Create AKS cluster and attach it to your workspace using AKS creation the commands in ray_aks_setup.yaml
3. Deploy Azure ML web services with following commmand (install azure ml cli v2 if neccessary)
 az online-endpoint create -f endpoint.yml
 az online-deployment create -f blue-deployment.yml
4. Update the environment variables with values in ray_service.yaml line 80-91 with values of a service principal to download model from your workspace (creating one and assign role if neccessary)
5. Run ray service deployment commands in ray_aks_setup.yaml 
6. If you update your code (ray_score.py), then you have to zip the folder script and push to ray repo. Update the line 15 in ray_aks_setup.yaml to point the working_dir to the zip file in your git repo



In [ ]:
import json
import urllib.request
import random
def score(X, tenant):
    # tenant = random.choices(["tenant1","tenant2", "tenant3","tenant4","tenant5","tenant6","tenant7", "tenant8","tenant9"])[0]

    len = random.randint(2,50)
    data = {"tenant":tenant, "data": X.head(len).to_numpy().tolist()}

    body = str.encode(json.dumps(data))

    url = ''
    api_key = '' # Replace this with the API key for the web service


    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(json.loads(result))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib
dataset = pd.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv")
X= dataset[['sepal_length','sepal_width','petal_length','petal_width']]


In [ ]:
score(X, "tenant13")